<a href="https://www.kaggle.com/code/mafiosoquasar/sales-forecasting-with-pytorch?scriptVersionId=209538807" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import argparse
import os
import random,numpy,pandas
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'

In [2]:
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.data
import torchvision
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import torchvision.transforms.functional as RF
from PIL import Image
import numpy as np
import random,cv2,pandas,os,numpy
import matplotlib.pyplot as plt
import matplotlib.animation as animation

In [3]:
seed = 999
print("Random Seed: ", seed)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

nz = 100
beta1 = 0.5
lr = 0.0001
batch_size=100
ngpu=1
ngf,nc = 3,3
ndf = 64

device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

Random Seed:  999


In [4]:
x = torch.load("/kaggle/input/encoder-weight-data/train_tensor.pt")[::200]

y=pandas.read_csv("/kaggle/input/rohlik-sales-forecasting-challenge-v2/sales_train.csv").sort_values(by=['date', 'warehouse'])
y=torch.from_numpy(numpy.nan_to_num(y['sales'].to_numpy().astype(numpy.float32))).reshape(len(y),1)[::200]


/tmp/ipykernel_23/3331271538.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  x = torch.load("/kaggle/input/encoder-weight-data/train_tensor.pt")[::200]


In [5]:
x.shape, y.shape, x

(torch.Size([20038, 1]),
 torch.Size([20038, 1]),
 tensor([[25.9468],
         [26.0216],
         [26.3887],
         ...,
         [29.4422],
         [27.3667],
         [30.7281]]))

In [6]:
train_x = torch.utils.data.DataLoader(x,batch_size=batch_size)
train_y = torch.utils.data.DataLoader(y,batch_size=batch_size)

In [7]:
def weights_init(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_normal_(m.weight.data, gain=0.02)
        
class RF_NET(nn.Module):
    def __init__(self):
        super().__init__()
        self.rafire = nn.Sequential(
            
            nn.Linear(1, 1524),
            nn.BatchNorm1d(1524),
            nn.ReLU(),

            nn.Linear(1524, 1324),
            nn.BatchNorm1d(1324),
            nn.ReLU(),

            nn.Linear(1324, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),

            nn.Linear(1024, 824),
            nn.BatchNorm1d(824),
            nn.ReLU(),

            nn.Linear(824, 524),
            nn.BatchNorm1d(524),
            nn.ReLU(),

            nn.Linear(524, 324),
            nn.BatchNorm1d(324),
            nn.ReLU(),

            nn.Linear(324, 124),
            nn.BatchNorm1d(124),
            nn.ReLU(),
            
            nn.Linear(124, 1)
        )

    def forward(self,x):
        #x,_=self.rafire_0(x)
        #print(x,x.shape)
        return self.rafire(x)

In [8]:
rf_net = RF_NET().type(torch.float32).to(device)
rf_net= nn.DataParallel(rf_net).to(device)
if (device.type == 'cuda') and (ngpu > 1):
    rf_net = nn.DataParallel(rf_net, list(range(ngpu)))
rf_net.apply(weights_init)

DataParallel(
  (module): RF_NET(
    (rafire): Sequential(
      (0): Linear(in_features=1, out_features=1524, bias=True)
      (1): BatchNorm1d(1524, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Linear(in_features=1524, out_features=1324, bias=True)
      (4): BatchNorm1d(1324, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU()
      (6): Linear(in_features=1324, out_features=1024, bias=True)
      (7): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (8): ReLU()
      (9): Linear(in_features=1024, out_features=824, bias=True)
      (10): BatchNorm1d(824, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (11): ReLU()
      (12): Linear(in_features=824, out_features=524, bias=True)
      (13): BatchNorm1d(524, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (14): ReLU()
      (15): Linear(in_features=524, out_features=324, bias

In [9]:
criterion = nn.L1Loss()

optimizer = optim.AdamW(rf_net.parameters(), lr=lr,betas=(beta1, 0.999))
scheduler=torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.86)

In [10]:
high_rf,i,j_r,k,z_w_=100000,0,0,0,[]
if os.path.exists(f"/kaggle/working/jsr_rf")==False:
    os.mkdir(f"/kaggle/working/jsr_rf")
    
correct,total=0,0

while(True):
    optimizer.zero_grad()
    for data,label in zip(train_x,train_y):
        #z_ = nn.Embedding(10, 3, dtype=torch.float64)
        #print(z_(torch.FloatTensor(data)))
        
        output = rf_net(data.to(device)).view(-1)
        err_real = criterion(output, label.to(device))
        err_real.backward()
        optimizer.step()
        #print(err_real.item())
        
    print(f"EPOCH : {i} LOSS_wl : {err_real.item()}")

    if len(z_w_)>=3:
        if len([True for i in range(1,4) if z_w_[len(z_w_)-i]<=z_w_[len(z_w_)-3] and z_w_[len(z_w_)-i]>=z_w_[len(z_w_)-4]])==3:
            z_w_,j_r=[],0
            print(f"lr_br:{optimizer.param_groups[0]['lr']}".upper())
            scheduler.step()
            print(f"lr_up:{optimizer.param_groups[0]['lr']}".upper())
            
    z_w_.append(err_real.item())
    if err_real.item()<high_rf:
            high_rf=err_real.item()
            torch.save(rf_net.state_dict(),f'/kaggle/working/jsr_rf/{err_real.item()}.pth')
    if i==10:
        break
    i+=1

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/modules/loss.py:101: UserWarning: Using a target size (torch.Size([100, 1])) that is different to the input size (torch.Size([100])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/opt/conda/lib/python3.10/site-packages/torch/nn/modules/loss.py:101: UserWarning: Using a target size (torch.Size([38, 1])) that is different to the input size (torch.Size([38])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


EPOCH : 0 LOSS_wl : 188.56044006347656
EPOCH : 1 LOSS_wl : 187.11549377441406
EPOCH : 2 LOSS_wl : 185.37448120117188
EPOCH : 3 LOSS_wl : 183.87326049804688
LR_BR:0.0001
LR_UP:8.6E-05
EPOCH : 4 LOSS_wl : 182.5398406982422
EPOCH : 5 LOSS_wl : 181.37518310546875
EPOCH : 6 LOSS_wl : 180.03208923339844
LR_BR:8.6E-05
LR_UP:7.396E-05
EPOCH : 7 LOSS_wl : 179.4379119873047
EPOCH : 8 LOSS_wl : 178.55271911621094
EPOCH : 9 LOSS_wl : 177.9174346923828
LR_BR:7.396E-05
LR_UP:6.36056E-05
EPOCH : 10 LOSS_wl : 177.2777099609375
